<a href="https://colab.research.google.com/github/Anson3208/Sentiment-Textmining-Analysis-Learning/blob/main/04_Social_Media_Sentiment_Analysis_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 - Social Media Sentiment Analysis Twitter


Naive Bayes will be adopted to perform Sentiment Analysis

# Large Movie Review Dataset

The Large Movie Review Dataset is a corpus of 50,000 movie reviews from IMDB that have been classified as either positive or negative. More information about the dataset can be found at https://ai.stanford.edu/~amaas/data/sentiment/. 

The following code downloads a copy of the Large Movie Review Dataset and saves it in a variable `imdb_corpus`. 

In [ ]:
import urllib.request, json 
imdb_corpus = []
with urllib.request.urlopen("https://storage.googleapis.com/wd13/IMDBReviewSent.txt") as url:
  for line in url.readlines():
    imdb_corpus.append(line.decode().split('\t'))

`imdb_corpus` is a list. Each element of the list is another list which stores a document and its label.

In [ ]:
# print the text and label of document 16
docid = 1
print(imdb_corpus[docid])

['positive', 'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are te

In [ ]:
# print the label of document 16
docid = 16
print(imdb_corpus[docid][0])

positive


In [ ]:
# print the text of document 16
docid = 16
print(imdb_corpus[docid][1])

Some films just simply should not be remade. This is one of them. In and of itself it is not a bad film. But it fails to capture the flavor and the terror of the 1963 film of the same title. Liam Neeson was excellent as he always is, and most of the cast holds up, with the exception of Owen Wilson, who just did not bring the right feel to the character of Luke. But the major fault with this version is that it strayed too far from the Shirley Jackson story in it's attempts to be grandiose and lost some of the thrill of the earlier film in a trade off for snazzier special effects. Again I will say that in and of itself it is not a bad film. But you will enjoy the friction of terror in the older version much more.



# Create a tokenizer

Write a function `tokenize` that takes a string and returns a list of tokens. 

In [ ]:
#Copying tokenizer from week 4 solution
import re
def tokenize(doc):
  emoti_list = [':)','(:',':(','):',':D','D:',':P','P:',':V','V:',':/','/:',':\\','\\:',':|','|:',
                ';)','(;',';(',');',';D','D;',';P','P;',';V','V;',';/','/;',';\\','\\;',';|','|;',
                ':-)','(-:',':-(',')-:',':-D','D-:',':-P','P-:',':-V','V-:',':-/','/-:',':-\\',
                '\\-:',':-|','|-:',';-)','(-;',';-(',')-;',';-D','D-;',';-P','P-;',';-V','V-;',
                ';-/','/-;',';-\\','\\-;',';-|','|-;']
  tokenizer_pattern = re.compile('|'.join([
      '|'.join([re.escape(e) for e in emoti_list]),
      "[A-Za-z]+(?:['-_\.][A-Za-z]+)?",
      '\.\.+'
      ]))
  tokens = tokenizer_pattern.findall(doc)
  for i in range(0,len(tokens)):
    if re.match('\.\.+',tokens[i]):
      tokens[i] = '..+'
    else:
      tokens[i] = tokens[i].lower()
  return(tokens)

# Import log function

In [ ]:
from math import log
log(1)

0.0

# Create a lexicon

Calculate sentiment scores for every token in the corpus, using the method discussed in class. Store these scores in a dictionary called `lexicon`. 

In [ ]:

corpus_positive_count = 0
corpus_negative_count = 0

token_positive_count = {}
token_negative_count = {}

unique_tokens = set()
lexicon = {}

for doc in imdb_corpus:
  label = doc[0]
  tokens = tokenize(doc[1])

#calculate corpus positive/negative count
  if label == 'positive':
    corpus_positive_count += 1
  else:
    corpus_negative_count +=1

#calculate token positive/negative count
  for token in set(tokens):
    unique_tokens.add(token)
    if label == 'positive':
      if token not in token_positive_count:
        token_positive_count [token] = 1
      else:
        token_positive_count [token] += 1
    else:
      if token not in token_negative_count:
        token_negative_count [token] = 1
      else:
        token_negative_count [token] += 1

#calculate score for lexicon
for token in unique_tokens:
  if token not in token_positive_count or token not in token_negative_count:
    continue
  else:
    lexicon[token] = log((token_positive_count[token]/corpus_positive_count)/(token_negative_count[token]/corpus_negative_count))

#Update 'not' to 0 from lexicon and adjust negation calculation in score message function
lexicon['not'] =0



In [ ]:
#Check lexicon
print(lexicon['movie'])
print(lexicon['good'])

-0.1694416703576676
-0.03189482684337694


# Create a score message function

Write a funciton `score_message` that takes a message `doc` and returns a sentiment score, using the method discussed in class. 

In [ ]:
def sentiment_score(document):
    """Calculate the sentiment score for a given document"""
    sentiment_score = log(corpus_positive_count/corpus_negative_count)
    tokens = tokenize(document)

    word_after_not = None                                   # the word after negation
    if 'not' in tokens:                                     # check if 'not' in tokens
        word_after_not = tokens.index('not') + 1            # position of the word after not in tokens

    for index, token in enumerate(tokens):
        if token in lexicon:
            if index != word_after_not:
                sentiment_score += lexicon[token]
            else:
                sentiment_score += lexicon[token] * -1      # Multiplied by -1 for the score adjusment of negation
            #print(token,lexicon[token],sentiment_score)
    #print('Total score', sentiment_score)    
    return sentiment_score

In [ ]:
# Check Sentiment Score
print(sentiment_score('It is amazing '))
print(sentiment_score('It is not amazing'))

1.2607217956781696
-1.252024365760766


# Live Twitter Data

To download live Twitter data, you will need a bearer token. To get a bearer token, you will need a devloper account. Go to https://developer.twitter.com/en to sign up.

Once you have your bearer token, create a new text file on your computer called `'twitter_bearer_token.txt'`. This file should contain a single line consiting of your bearer token and nothing else (no white space, no new lines, just the token). 

The code below will prompt you to upload a file from your computer. Upload 'twitter_bearer_token.txt'. Your bearer token will be saved to a string twitter_bearer_token.

In [ ]:
from google.colab import files
uploaded_files = files.upload()
twitter_bearer_token = uploaded_files['twitter_bearer_token.txt'].decode()

Saving twitter_bearer_token.txt to twitter_bearer_token.txt


Below we define the function get_tweets. This function returns tweets matching a specificed query. It takes the following parameters:

query your search query
bearer_token your Twitter API bearer token
number_of_tweets the number of tweets you want to return
This code uses the Twitter API's Recent Search endpoint. See https://developer.twitter.com/en/docs/twitter-api/tweets/search/introduction for documentation. This endpoint only provides access to tweets from the past week.

In [ ]:
import requests
def get_tweets(query,bearer_token,number_of_tweets):
  tweets = []
  next_token = None
  while len(tweets)<number_of_tweets:
    response = requests.get(
        url = 'https://api.twitter.com/2/tweets/search/recent',
        params = {
          'query':query,
          'next_token':next_token},
        headers = {'authorization' : 'bearer '+bearer_token} 
        )
    response_json = response.json()
    for tweet in response_json['data']:
      tweets.append(tweet)
    if 'next_token' not in response_json['meta']:
      break
    next_token = response_json['meta']['next_token']
  return(tweets)

The following code downloads 500 recent tweets containing the words "nike" or "adidas" and saves them in tweets.

In [ ]:
tweets = get_tweets('nike OR adidas',twitter_bearer_token,500)

Each tweet is represented by a dictionary with three fields:

edit_history_tweet_ids
id
text
It is possible to download additional fields. See the API documentation referenced above.

In [ ]:
tweets[200]

{'edit_history_tweet_ids': ['1624532227697655809'],
 'id': '1624532227697655809',
 'text': '@alinvenex @boraejk @kinetix_tr @PUMATurkiye @lumberjackTR @Nike @ninewestturkiye @PolarisTurkiye @adidas @DockerSbyGerli @butigocom @lottosport @Reebok @uspoloassntr @Vans İNSANLARA YIPRANMIŞ YIRTIK AYAKKABILAR YOLLUYORLARMIŞ AYAKKABI MARKALARINI ETİKETLEYİP DESTEK İSTİYORUZ @kinetix_tr @PUMATurkiye @lumberjackTR @Nike @ninewestturkiye @PolarisTurkiye @adidas @DockerSbyGerli @butigocom @lottosport @Reebok @uspoloassntr @Vans'}

The following code allows you to download the tweets into a json file that you can save to your computer.

# Analyze Twitter Data

In [ ]:
import json
with open('tweets.json','w') as f:
  f.write(json.dumps(tweets))
files.download('tweets.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Choose a topic and perform some analysis of tweets related to that topic. Use some (or all) of the statistics we discussed in class:
* Number of Tweets
* Share of Voice
* Positive Percent
* Negative Percent
* Net Positive Percent

## Topic: Explore Nike and Adidas on Twitter

In [ ]:
#Subset text in tweet for analysis
tweets_text = [tweet['text'] for tweet in tweets]

#test
tweets_text[0]

'📌Adidas Live @Gman10312619 @Alphie40750663 @chicktheboy @Thorste49651905 @hk_azuki @sipcocacola @dika48470938 @TheAntzNest @aleksandrovn696 @nic_pic10 @MoCryptoFTW @portia15805644 https://t.co/uvyU9oqweG'

First, we count how many tweets are related to Nike and Adidas, and how many of those are positive and negative.

In [ ]:
#Count how many tweets for Nike and Adidas and sentiment scores

nike_total_count = 0
nike_positive_count = 0
nike_negative_count = 0
adidas_total_count = 0
adidas_positive_count = 0
adidas_negative_count = 0

for tweet in tweets_text:
  tweet_sentiment_score = sentiment_score(tweet)      #calculate sentiment score for each iteration
  tweet_tokens = set(tokenize(tweet))                 #tokenize tweet
  if 'nike' in tweet_tokens:                          #calculate for nike total_count
    nike_total_count +=1
    if tweet_sentiment_score >=0:                     #calculate nike positive count    
      nike_positive_count +=1                           
    else:
      nike_negative_count +=1                         #calculate nike negative count
  elif 'adidas' in tweet_tokens:
    adidas_total_count +=1
    if tweet_sentiment_score >=0:
      adidas_positive_count +=1
    else:
      adidas_negative_count +=1




In [ ]:
#Number of Tweets for Nike and Adidas
print('nike_total_count:', nike_total_count)
print('nike_positive_count:',nike_positive_count)
print('nike_negative_count:',nike_negative_count)
print('adidas_total_count:',adidas_total_count)
print('adidas_positive_count:',adidas_positive_count)
print('adidas_negative_count:',adidas_negative_count)

nike_total_count: 188
nike_positive_count: 121
nike_negative_count: 67
adidas_total_count: 269
adidas_positive_count: 192
adidas_negative_count: 77


We then calculate the following summary statistics:
* Number of Tweets
* Share of Voice
* Positive Percent
* Negative Percent
* Net Positive Percent

In [ ]:
nike_corpus_summary = {
    'NTweets' : nike_total_count,
    'ShareofVoice' : 100*nike_total_count/(nike_total_count+adidas_total_count),
    'PositivePct' : 100*nike_positive_count/nike_total_count,
    'NegativePct' : 100*nike_negative_count/nike_total_count,
    'NetPositivePct' : 100*(nike_positive_count-nike_negative_count)/nike_total_count
}
adidas_corpus_summary = {
    'NTweets' : adidas_total_count,
    'ShareofVoice' : 100*adidas_total_count/(nike_total_count+adidas_total_count),
    'PositivePct' : 100*adidas_positive_count/adidas_total_count,
    'NegativePct' : 100*adidas_negative_count/adidas_total_count,
    'NetPositivePct' : 100*(adidas_positive_count-adidas_negative_count)/adidas_total_count
}

We display these summary statistics in a table. 

In [ ]:
print("")
print(' | '+
      'Brand          | '+
      '# Tweets       | '+
      'Share of Voice | '+
      'Positive %     | '+
      'Negative %     | '+
      'Net Positive % | ')
print("")
print((" | "+
      "Adidas         | "+
      "{NTweets:5.0f}          | "+
      "{ShareofVoice:5.2f}%         | "+
      "{PositivePct:2.2f}%         | "+
      "{NegativePct:2.2f}%         | "+
      "{NetPositivePct:2.2f}%         | ").format(**adidas_corpus_summary))
print((" | "+
      "Nike           | "+
      "{NTweets:5.0f}          | "+
      "{ShareofVoice:5.2f}%         | "+
      "{PositivePct:2.2f}%         | "+
      "{NegativePct:2.2f}%         | "+
      "{NetPositivePct:2.2f}%         | ").format(**nike_corpus_summary))
print("")


 | Brand          | # Tweets       | Share of Voice | Positive %     | Negative %     | Net Positive % | 

 | Adidas         |   269          | 58.86%         | 71.38%         | 28.62%         | 42.75%         | 
 | Nike           |   188          | 41.14%         | 64.36%         | 35.64%         | 28.72%         | 

